In [1]:
import os, sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("K:/MachineLearning/MultiLevelClassifier/loan_prediction_dataset.csv")

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,product_1,product_2,product_3,product_4
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,1,0,0,1
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,1,0,0,1
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,0,1,1,1
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,0,1,1,1
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,1,1,1,1


In [4]:
# performing data cleaning
df.dropna(inplace=True)

In [5]:
#label encoding categorical values
label_encoder_dict = defaultdict(LabelEncoder)
df[['Gender', 'Married','Dependents', 'Education', 'Self_Employed', 'Property_Area']] = df[['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']].apply(lambda x: label_encoder_dict[x.name].fit_transform(x))

In [6]:
#scalling some of the columns
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
df[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 'Loan_Amount_Term']] = mms.fit_transform(df[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 'Loan_Amount_Term']])

In [7]:
#getting dependent and independent columns
customer_information = df.iloc[:,1:11]
target_columns = df.iloc[:,12:]

In [8]:
X_train, X_test, y_train,y_test = train_test_split(customer_information, target_columns, test_size=0.2)

In [9]:
#defining multilabel classifier
classifier = BinaryRelevance(classifier = LogisticRegression(solver='liblinear', random_state=0),
                             require_dense = [False, True])
classifier.fit(X_train, y_train)

BinaryRelevance(classifier=LogisticRegression(random_state=0,
                                              solver='liblinear'),
                require_dense=[False, True])

In [10]:
#to validate the probabilities, lets check with 1 row only
test_row = X_test[4:5]

In [19]:
#predicting probabilities
probabilities = classifier.predict_proba(test_row)
probability = probabilities.data[0]
target_list = target_columns.columns
proba_dict = dict(zip(target_list, probability))

In [20]:
proba_dict

{'product_1': 0.5532364225367716,
 'product_2': 0.5983419740368048,
 'product_3': 0.5553127581903295,
 'product_4': 0.479090846043101}